In [556]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
random.set_seed(2)
#import nltk as nltk
#nltk.download('vader_lexicon')
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#analyzer = SentimentIntensityAnalyzer()
#from nltk.corpus import stopwords, reuters
import hvplot.pandas
%matplotlib inline

In [557]:
load_dotenv()

True

In [558]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [559]:
api_key = os.getenv("news_api")
#print(api_key)

In [560]:
newsapi = NewsApiClient(api_key=api_key)

In [561]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["CVX", "AAPL", "GOOGL", "FB"]
nasdaq_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
nasdaq_ticker.dropna(inplace=True)
nasdaq_ticker.head(5)
#apple_ticker.dropna(inplace=True)

AAPL                                        \
                             open      high     low     close    volume   
time                                                                      
2019-01-02 00:00:00-05:00  154.89  158.8500  154.23  157.5225  27803621   
2019-01-03 00:00:00-05:00  143.98  145.7200  142.00  142.2900  82418448   
2019-01-04 00:00:00-05:00  144.53  148.5499  143.80  148.1200  51267225   
2019-01-07 00:00:00-05:00  148.70  148.8300  145.90  147.9100  49982480   
2019-01-08 00:00:00-05:00  149.56  151.8200  148.52  150.6300  36592406   

                               CVX                                       FB  \
                              open    high     low   close   volume    open   
time                                                                          
2019-01-02 00:00:00-05:00  107.340  111.68  107.10  110.50  4901089  128.99   
2019-01-03 00:00:00-05:00  111.055  111.29  108.07  108.78  5048448  134.69   
2019-01-04 00:00:00-05:00  110.280  110.99  109.47  110.74  5839159  134.01   
2019-01-07 00:00:00-05:00  111.060  112.93  109.92  112.35  4566504  137.45   
2019-01-08 00:00:00-05:00  113.400  113.79  111.67  111.85  4371673  139.89   

                                                                 GOOGL  \
                             high     low     close    volume     open   
time                                                                     
2019-01-02 00:00:00-05:00  137.51  128.56  135.4100  25465924  1027.20   
2019-01-03 00:00:00-05:00  137.17  131.12  131.8902  18011900  1050.67   
2019-01-04 00:00:00-05:00  138.00  133.75  137.9400  23927539  1041.63   
2019-01-07 00:00:00-05:00  138.87  135.91  138.2300  16261533  1080.97   
2019-01-08 00:00:00-05:00  143.14  139.54  142.5843  22239823  1084.77   

                                                                  
                              high        low     close   volume  
time                                                              
2019-01-02 00:00:00-05:00  1060.79  1025.2750  1053.240  1346501  
2019-01-03 00:00:00-05:00  1066.26  1022.3700  1026.505  1546910  
2019-01-04 00:00:00-05:00  1080.00  1036.8600  1076.550  1792417  
2019-01-07 00:00:00-05:00  1082.70  1062.6400  1076.010  1872050  
2019-01-08 00:00:00-05:00  1093.35  1068.3486  1085.510  1402631

In [562]:
tech_df = pd.DataFrame()
tech_df["Chevron"] = nasdaq_ticker["CVX"]["close"]
tech_df["Apple"] = nasdaq_ticker["AAPL"]["close"]
tech_df["Google"] = nasdaq_ticker["GOOGL"]["close"]
tech_df["Facebook"] = nasdaq_ticker["FB"]["close"]
#apple_df = apple_df.pct_change().dropna()
tech_df.index = tech_df.index.date 
tech_df.dropna(inplace=True)
tech_df.head()

,Chevron,Apple,Google,Facebook
2019-01-02,110.50,157.5225,1053.240,135.4100
2019-01-03,108.78,142.2900,1026.505,131.8902
2019-01-04,110.74,148.1200,1076.550,137.9400
2019-01-07,112.35,147.9100,1076.010,138.2300
2019-01-08,111.85,150.6300,1085.510,142.5843


In [563]:
tech_returns = tech_df.pct_change().dropna()
tech_returns.head()

,Chevron,Apple,Google,Facebook
2019-01-03,-0.015566,-0.096700,-0.025384,-0.025994
2019-01-04,0.018018,0.040973,0.048753,0.045870
2019-01-07,0.014539,-0.001418,-0.000502,0.002102
2019-01-08,-0.004450,0.018390,0.008829,0.031500
2019-01-09,0.012114,0.018190,-0.002110,0.011682


In [564]:
csv_path = Path("../csvs/Book1.csv/")
oil_data = pd.read_csv(csv_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
oil_data.sort_index(ascending = True, inplace = True)
oil_data.head()

,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87


In [565]:
oil = pd.DataFrame()
oil["DPB"] = oil_data["Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"]

In [566]:
oil.head()

,DPB
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87


In [567]:
csv_path = Path("../csvs/covid_cases.csv/")
covid_data = pd.read_csv(csv_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
##covid_data.sort_index(ascending = True, inplace = True)
covid_data.tail()

,New Cases,7-Day Moving Avg,Historic Cases
Date,,,
2020-01-26,3,6,0
2020-01-25,3,5,0
2020-01-24,1,5,0
2020-01-23,2,5,0
2020-01-22,33,4,0


In [568]:
covid_df = pd.DataFrame()
covid_df["US_Covid_Cases"] = covid_data["New Cases"]
#covid_df["Date"] = covid_data["Date"]
covid_df.head()

,US_Covid_Cases
Date,
2021-07-02,15555
2021-07-01,17911
2021-06-30,14353
2021-06-29,14823
2021-06-28,8669


In [569]:
df = tech_df.join(oil, how="inner")
df.head()

,Chevron,Apple,Google,Facebook,DPB
2019-01-02,110.50,157.5225,1053.240,135.4100,46.31
2019-01-03,108.78,142.2900,1026.505,131.8902,46.92
2019-01-04,110.74,148.1200,1076.550,137.9400,47.76
2019-01-07,112.35,147.9100,1076.010,138.2300,48.27
2019-01-08,111.85,150.6300,1085.510,142.5843,49.58


In [570]:
data = covid_df.join(df, how="inner")
#data = data.join(vaccine_df, how="inner")
data.sort_index(inplace=True)
data.head()

,US_Covid_Cases,Chevron,Apple,Google,Facebook,DPB
2020-01-22,33,112.910,317.7000,1483.56,221.290,56.76
2020-01-23,2,113.110,319.2899,1484.02,219.670,55.51
2020-01-24,1,111.855,318.3300,1466.05,217.929,54.09
2020-01-27,1,110.370,308.9600,1431.43,214.870,53.09
2020-01-28,1,111.120,317.7600,1449.61,217.790,53.33


In [571]:
data.shape[0]*.7

252.7

In [572]:
data.iloc[253]

US_Covid_Cases    121277.00
Chevron               90.93
Apple                142.90
Google              1894.28
Facebook             278.00
DPB                   52.78
Name: 2021-01-25 00:00:00, dtype: float64

In [573]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [574]:
window_size = 3
feature_column = 1
target_column = 5
X, y = window_data(data, window_size, feature_column, target_column)
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[112.91  113.11  111.855]
 [113.11  111.855 110.37 ]
 [111.855 110.37  111.12 ]
 [110.37  111.12  110.39 ]
 [111.12  110.39  111.42 ]] 

y sample values:
[[53.09]
 [53.33]
 [53.29]
 [52.19]
 [51.58]]


In [575]:
print(X)

[[112.91  113.11  111.855]
 [113.11  111.855 110.37 ]
 [111.855 110.37  111.12 ]
 ...
 [106.06  106.4   106.26 ]
 [106.4   106.26  107.14 ]
 [106.26  107.14  107.31 ]]


In [576]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]
X_train

array([[112.91  , 113.11  , 111.855 ],
       [113.11  , 111.855 , 110.37  ],
       [111.855 , 110.37  , 111.12  ],
       [110.37  , 111.12  , 110.39  ],
       [111.12  , 110.39  , 111.42  ],
       [110.39  , 111.42  , 107.11  ],
       [111.42  , 107.11  , 106.25  ],
       [107.11  , 106.25  , 106.86  ],
       [106.25  , 106.86  , 110.29  ],
       [106.86  , 110.29  , 109.51  ],
       [110.29  , 109.51  , 108.95  ],
       [109.51  , 108.95  , 109.79  ],
       [108.95  , 109.79  , 111.21  ],
       [109.79  , 111.21  , 112.05  ],
       [111.21  , 112.05  , 111.6924],
       [112.05  , 111.6924, 110.07  ],
       [111.6924, 110.07  , 110.25  ],
       [110.07  , 110.25  , 110.73  ],
       [110.25  , 110.73  , 109.8   ],
       [110.73  , 109.8   , 109.    ],
       [109.8   , 109.    , 104.72  ],
       [109.    , 104.72  , 100.73  ],
       [104.72  , 100.73  ,  98.05  ],
       [100.73  ,  98.05  ,  94.085 ],
       [ 98.05  ,  94.085 ,  93.33  ],
       [ 94.085 ,  93.33 

In [577]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [578]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.99660326]
  [1.        ]
  [0.99662746]]

 [[1.        ]
  [0.97868546]
  [0.97094431]]

 [[0.97868546]
  [0.95346467]
  [0.9839156 ]]

 [[0.95346467]
  [0.96620245]
  [0.97129021]]

 [[0.96620245]
  [0.95380435]
  [0.98910412]]] 

X_test sample values:
[[[0.69887908]
  [0.64164402]
  [0.64856451]]

 [[0.64164402]
  [0.63688859]
  [0.63472847]]

 [[0.63688859]
  [0.62330163]
  [0.60774818]]

 [[0.62330163]
  [0.59680707]
  [0.58855067]]

 [[0.59680707]
  [0.57795516]
  [0.60048426]]]


In [579]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [580]:
model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [581]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [582]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_48 (LSTM)               (None, 3, 5)              140       
_________________________________________________________________
dropout_48 (Dropout)         (None, 3, 5)              0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 3, 5)              220       
_________________________________________________________________
dropout_49 (Dropout)         (None, 3, 5)              0         
_________________________________________________________________
lstm_50 (LSTM)               (None, 5)                 220       
_________________________________________________________________
dropout_50 (Dropout)         (None, 5)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)               

In [583]:
model.fit(X_train, y_train, epochs=15, shuffle=False, batch_size=1, verbose=1)

Epoch 1/15
250/250 [==============================] - 5s 4ms/step - loss: 0.1309
Epoch 2/15
250/250 [==============================] - 1s 3ms/step - loss: 0.0236
Epoch 3/15
250/250 [==============================] - 1s 3ms/step - loss: 0.0207
Epoch 4/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0185
Epoch 5/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0169
Epoch 6/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0168
Epoch 7/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0158
Epoch 8/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0164
Epoch 9/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0121
Epoch 10/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0134
Epoch 11/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0111
Epoch 12/15
250/250 [==============================] - 1s 4ms/step - loss: 0.0109
Epoch 13/15
250/250 [====

In [584]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 1s 2ms/step - loss: 0.0185


0.01853838749229908

In [585]:
predicted = model.predict(X_test)

In [586]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [587]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, index = data.index[-len(real_prices): ])
stocks.head(100)

,Real,Predicted
2021-01-25,52.78,47.153328
2021-01-26,52.61,46.753361
2021-01-27,52.81,46.627998
2021-01-28,52.26,46.408039
2021-01-29,52.16,46.183479
...,...,...
2021-06-10,70.34,49.839958
2021-06-11,71.00,49.895531
2021-06-14,70.94,49.829659
2021-06-15,72.06,49.862411


In [588]:
stocks.hvplot.line(xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [589]:
stocks.hvplot.line(xlabel="Date", ylabel="Price")

:NdOverlay   [Variable]
   :Curve   [index]   (value)